<a href="https://colab.research.google.com/github/ARt2406/June_test/blob/main/HomeWork_Modules_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:

#!mkdir my_package

#%%writefile my_package/__init__.py

#%%writefile my_package/products.py


class InvalidPriceError(Exception):
  def __init__(self, message='Price must more then zero'):
    self.message = message
    super().__init__(self.message)

class InvalidQuantityError(Exception):
  def __init__(self, message='InvalidQuantityError!!!!!!!'):
    self.message = message
    super().__init__(self.message)

class LoggingMixin():
  def log(self, message):
    print(f'[LOG]: {message}')


class Product(LoggingMixin):
    def __init__(self, name, price, description):
      if price <= 0:
        self.log(f'Zero price')
        raise InvalidPriceError
      self.name = name
      self.price = price
      self.description = description
      #self.log(f'Add new product')

    def __str__(self):
        return f'{self.name} - ${self.price} UAH'



Overwriting my_package/products.py


In [51]:



%%writefile my_package/discont.py
from my_package.products import LoggingMixin

class Discount:
    def apply(self, price):
        pass


class PercentageDiscount(Discount):
    def __init__(self, percentage: float | int = 0.1):
        if 0 <= percentage <= 1:
            self.percentage = percentage
        else:
            self.percentage = 0

    def apply(self, price: float | int):
        return price * (1 - self.percentage)



class FixedAmountDiscount(Discount, LoggingMixin):
    def __init__(self, amount: float | int = 0):
        if amount < 0:
            amount = 0
        self.amount = amount
        self.log(f'FixedAmountDiscount is {self.amount}')

    def apply(self, price: float | int):
        if price < self.amount:
            return 0
        return price - self.amount



class DiscountMixin():
    def apply_discount(self, discount: Discount):
        # if self.products not exists
        if hasattr(self, 'products'):
            for product in self.products:
                product.price = discount.apply(product.price)




Overwriting my_package/discont.py


In [54]:

%%writefile my_package/pay.py
from my_package.discont import DiscountMixin


class PaymentProcessor:
    def pay(self, amount):
        pass


class CreditCardProcessor(PaymentProcessor):
    def __init__(self, card_number, card_holder, cvv, expiry_date):
        self.card_number = card_number
        self.card_holder = card_holder
        self.cvv = cvv
        self.expiry_date = expiry_date

    def pay(self, amount):
        print(f'Paying ${amount} with credit card {self.card_number}')


class PayPalProcessor(PaymentProcessor):
    def __init__(self, email):
        self.email = email

    def pay(self, amount):
        print(f'Paying ${amount} with PayPal account {self.email}')


class BankTransferProcessor(PaymentProcessor):
    def __init__(self, account_number, account_holder):
        self.account_number = account_number
        self.account_holder = account_holder

    def pay(self, amount):
        print(f'Paying ${amount} with bank transfer from account {self.account_number}')


class Cart(DiscountMixin):
    def __init__(self):
        self.products = {}

    def add_product(self, product, quantity):
      if quantity <= 0:
        raise InvalidQuantityError
      self.products[product] = self.products.get(product, 0) + quantity

    def total_cost(self):
        return sum(product.price * quantity for product, quantity in self.products.items())

    def pay(self, payment_processor: PaymentProcessor):
        payment_processor.pay(self.total_cost())

    def __str__(self):
        return '\n'.join(f'{product} x {quantity} = {product.price * quantity} UAH'
                         for product, quantity in self.products.items())




Overwriting my_package/pay.py


In [55]:
from my_package.products import Product, InvalidQuantityError, LoggingMixin
from my_package.discont import PercentageDiscount, FixedAmountDiscount, DiscountMixin
from my_package.pay import CreditCardProcessor, PayPalProcessor, BankTransferProcessor

def main():
    # Creating instances of the Product class
    product1 = Product("Laptop", 1500.00, "A high-end gaming laptop")
    product2 = Product("Mouse", 50.00, "A wireless mouse")
    product3 = Product("Keyboard", 100.00, "A mechanical keyboard")

    # Creating an instance of the Cart class and adding products
    cart = Cart()
    cart.add_product(product1, 1)
    cart.add_product(product2, 2)
    cart.add_product(product3, 1)

    print(cart)
    print("Total cost:", cart.total_cost())

    # Applying different types of discounts
    percentage_discount = PercentageDiscount(0.5)
    fixed_amount_discount = FixedAmountDiscount(20)

    cart.apply_discount(percentage_discount)
    print(cart)
    print("Total cost after percentage discount:", cart.total_cost())

    cart.apply_discount(fixed_amount_discount)
    print(cart)
    print("Total cost after fixed amount discount:", cart.total_cost())

    # Using different payment systems
    credit_card_processor = CreditCardProcessor("1234-5678-9876-5432", "John Doe", "123", "12/25")
    paypal_processor = PayPalProcessor("john.doe@example.com")
    bank_transfer_processor = BankTransferProcessor("987654321", "John Doe")

    cart.pay(credit_card_processor)
    cart.pay(paypal_processor)
    cart.pay(bank_transfer_processor)


if __name__ == "__main__":
    main()

Laptop - $1500.0 UAH x 1 = 1500.0 UAH
Mouse - $50.0 UAH x 2 = 100.0 UAH
Keyboard - $100.0 UAH x 1 = 100.0 UAH
Total cost: 1700.0
[LOG]: FixedAmountDiscount is 20
Laptop - $750.0 UAH x 1 = 750.0 UAH
Mouse - $25.0 UAH x 2 = 50.0 UAH
Keyboard - $50.0 UAH x 1 = 50.0 UAH
Total cost after percentage discount: 850.0
Laptop - $730.0 UAH x 1 = 730.0 UAH
Mouse - $5.0 UAH x 2 = 10.0 UAH
Keyboard - $30.0 UAH x 1 = 30.0 UAH
Total cost after fixed amount discount: 770.0
Paying $770.0 with credit card 1234-5678-9876-5432
Paying $770.0 with PayPal account john.doe@example.com
Paying $770.0 with bank transfer from account 987654321
